In [ ]:
import timeimport numpy as npimport tensorflow as tffrom q1_softmax import softmaxfrom q1_softmax import cross_entropy_lossfrom model import Modelfrom utils.general_utils import get_minibatchesclass Config(object):"""Holds model hyperparams and data information.The config class is used to store various hyperparameters and datasetinformation parameters. Model objects are passed a Config() object atinstantiation."""n_samples = 1024n_features = 100n_classes = 5batch_size = 64n_epochs = 50lr = 1e-4class SoftmaxModel(Model):"""Implements a Softmax classifier with cross-entropy loss."""def add_placeholders(self):"""Generates placeholder variables to represent the input tensors.These placeholders are used as inputs by the rest of the model buildingand will be fed data during training.Adds following nodes to the computational graphinput_placeholder: Input placeholder tensor of shape(batch_size, n_features), type tf.float32labels_placeholder: Labels placeholder tensor of shape(batch_size, n_classes), type tf.int32Add these placeholders to self as the instance variablesself.input_placeholderself.labels_placeholder"""### YOUR CODE HEREself.input_placeholder = tf.placeholder(dtype=tf.float32, shape=[self.config.batch_size, self.config.n_features])self.labels_placeholder = tf.placeholder(dtype=tf.float32, shape=[self.config.bathc_size, self.config.n_classes])### END YOUR CODEdef create_feed_dict(self, inputs_batch, labels_batch=None):"""Creates the feed_dict for training the given step.A feed_dict takes the form of:feed_dict = {<placeholder>: <tensor of values to be passed for placeholder>,....}If label_batch is None, then no labels are added to feed_dict.Hint: The keys for the feed_dict should be the placeholdertensors created in add_placeholders.Args:inputs_batch: A batch of input data.labels_batch: A batch of label data.Returns:feed_dict: The feed dictionary mapping from placeholders to values."""### YOUR CODE HEREif labels_batch is None:feed_dict = {self.input_placeholder: inputs_batch}else:feed_dict = {self.input_placeholder: inputs_batchself.labels_placeholder: labels_batch}### END YOUR CODEreturn feed_dictdef add_prediction_op(self):"""Adds the core transformation for this model which transforms a batch of inputdata into a batch of predictions. In this case, the transformation is a linear layer plus asoftmax transformation:y = softmax(Wx + b)Hint: Make sure to create tf.Variables as needed.Hint: For this simple use-case, it's sufficient to initialize both weights Wand biases b with zeros.Args:input_data: A tensor of  shape (batch_size, n_features).Returns:pred: A tensor of shape (batch_size, n_classes)"""### YOUR CODE HEREW = tf.Variable(tf.zeros(shape=[self.config.n_features, self.config.n_classes], dtype=tf.float32), dtype=tf.float32, name='W')b = tf.Variable(tf.zeros(shape=[self.config.n_classes], dtype=tf.float32), dtype=tf.float32, name='b')x = tf.matmul(self.input_placeholder, W) + bpred = softmax(x)### END YOUR CODEreturn preddef add_loss_op(self, pred):"""Adds cross_entropy_loss ops to the computational graph.Hint: Use the cross_entropy_loss function we defined. This should be a veryshort function.Args:pred: A tensor of shape (batch_size, n_classes)Returns:loss: A 0-d tensor (scalar)"""### YOUR CODE HEREloss = cross_entropy_loss(tf.one_hot(self.labels_placeholder, ), pred)### END YOUR CODEreturn lossdef add_training_op(self, loss):"""Sets up the training Ops.Creates an optimizer and applies the gradients to all trainable variables.The Op returned by this function is what must be passed to the`sess.run()` call to cause the model to train. Seehttps://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizerfor more information.Hint: Use tf.train.GradientDescentOptimizer to get an optimizer object.Calling optimizer.minimize() will return a train_op object.Args:loss: Loss tensor, from cross_entropy_loss.Returns:train_op: The Op for training."""### YOUR CODE HERE### END YOUR CODEreturn train_opdef run_epoch(self, sess, inputs, labels):"""Runs an epoch of training.Args:sess: tf.Session() objectinputs: np.ndarray of shape (n_samples, n_features)labels: np.ndarray of shape (n_samples, n_classes)Returns:average_loss: scalar. Average minibatch loss of model on epoch."""n_minibatches, total_loss = 0, 0for input_batch, labels_batch in get_minibatches([inputs, labels], self.config.batch_size):n_minibatches += 1total_loss += self.train_on_batch(sess, input_batch, labels_batch)return total_loss / n_minibatchesdef fit(self, sess, inputs, labels):"""Fit model on provided data.Args:sess: tf.Session()inputs: np.ndarray of shape (n_samples, n_features)labels: np.ndarray of shape (n_samples, n_classes)Returns:losses: list of loss per epoch"""losses = []for epoch in range(self.config.n_epochs):start_time = time.time()average_loss = self.run_epoch(sess, inputs, labels)duration = time.time() - start_timeprint 'Epoch {:}: loss = {:.2f} ({:.3f} sec)'.format(epoch, average_loss, duration)losses.append(average_loss)return lossesdef __init__(self, config):"""Initializes the model.Args:config: A model configuration object of type Config"""self.config = configself.build()def test_softmax_model():"""Train softmax model for a number of steps."""config = Config()# Generate random data to train the model onnp.random.seed(1234)inputs = np.random.rand(config.n_samples, config.n_features)labels = np.zeros((config.n_samples, config.n_classes), dtype=np.int32)labels[:, 0] = 1# Tell TensorFlow that the model will be built into the default Graph.# (not required but good practice)with tf.Graph().as_default():# Build the model and add the variable initializer Opmodel = SoftmaxModel(config)init = tf.global_variables_initializer()# If you are using an old version of TensorFlow, you may have to use# this initializer instead.# init = tf.initialize_all_variables()# Create a session for running Ops in the Graphwith tf.Session() as sess:# Run the Op to initialize the variables.sess.run(init)# Fit the modellosses = model.fit(sess, inputs, labels)# If Ops are implemented correctly, the average loss should fall close to zero# rapidly.assert losses[-1] < .5print "Basic (non-exhaustive) classifier tests pass"if __name__ == "__main__":test_softmax_model()